In [2]:
# pip install ultralytics
import ultralytics
ultralytics.__version__

import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [3]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
tracker=Tracker()
count=0
cap=cv2.VideoCapture('trafficvideo.mp4')
down={}
up={}

counter_down=[]
counter_up=[]

In [4]:
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    a = a.detach().cpu().numpy() 
    px=pd.DataFrame(a).astype("float")
    #print(px)

    list=[]
             
    for index,row in px.iterrows():
#        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
            #print(c)

    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        red_line_y=198
        blue_line_y=268   
        offset = 7
        
  
        ## condition for counting the cars which are entering from red line and exiting from blue line
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
          down[id]=cy   
        if id in down:
           if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):         
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
             #counter+=1
             counter_down.append(id) 

        ## condition for counting the cars which are entering from blue line and exiting from red line
        # if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
        #   up[id]=cy   
        # if id in up:
        #    if red_line_y < (cy + offset) and red_line_y > (cy - offset):         
        #      cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
        #      cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
        #      #counter+=1
        #      counter_up.append(id)  




    
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    blue_color = (255, 0, 0)  # (B, G, R)
    green_color = (0, 255, 0)  # (B, G, R)  

    cv2.line(frame,(172,198),(774,198),red_color,3)  #  starting cordinates and end of line cordinates
    # cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    cv2.line(frame,(8,268),(927,268),blue_color,3)  # seconde line
    # cv2.putText(frame,('blue line'),(8,268),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    downwards = (len(counter_down))//4
    cv2.putText(frame,('Vehicles going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    

    
    upwards = (len(counter_up))//4
    cv2.putText(frame,('Vehicles going up - ')+ str(upwards),(60,60),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)  

    cv2.imshow("frames", frame)
    # if cv2.waitKey(1)&0xFF==27:
    #     break
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 3 cars, 1 train, 1 truck, 147.1ms
Speed: 5.3ms preprocess, 147.1ms inference, 2.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 truck, 87.8ms
Speed: 2.0ms preprocess, 87.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 truck, 99.6ms
Speed: 0.0ms preprocess, 99.6ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 train, 1 truck, 91.1ms
Speed: 1.0ms preprocess, 91.1ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 train, 1 truck, 90.9ms
Speed: 0.0ms preprocess, 90.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 1 truck, 93.7ms
Speed: 5.9ms preprocess, 93.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 1 truck, 91.6ms
Speed: 4.0ms preprocess, 91.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 1 truck, 95.0ms
Spee